In [19]:
import pandas as pd

In [20]:
import numpy as np

In [21]:
import lofargeo

In [67]:
import glob

## Read in antenna data from LOFAR svn

In [240]:
data = pd.DataFrame();
for stationfile in glob.glob('/Users/dijkema/opt/lofar/src/MAC/Deployment/data/Coordinates/ETRF_FILES/*/*.csv'):
    stationframe = pd.read_csv(stationfile)
    stationframe["STATIONNAME"] = stationfile.split("/")[-2]
    data = pd.concat([data, stationframe], ignore_index=True)

Separate the 'NAME' column into a type and a number:

Add a column `TYPE` with `LBA` and `HBA`:

In [179]:
data["TYPE"] = data["NAME"].str[0]+"BA"

For the centers (`CLBA`, `CHBA`, `CHBA0`, `CHBA1`), type is the same as the name:

In [180]:
data.loc[data["NAME"].str[0]=="C", "TYPE"] = data.loc[data["NAME"].str[0]=="C", "NAME"]

In [186]:
pd.value_counts(data[data["TYPE"].str[0]=="C"]["TYPE"])

CLBA     51
CHBA     45
CHBA0    24
CHBA1    24
CHBA      6
Name: TYPE, dtype: int64

Add a column `ANTENNANUMBER` with the numeric part of `NAME`:

In [182]:
data["ANTENNANUMBER"] = 0

In [188]:
data.loc[data["NAME"].str[0]!="C", "TYPE"] = data.loc[data["NAME"].str[0]!="C", "NAME"].str[1:]

Use the `lofargeo` package from Michiel Brentjens to convert from ETRS x, y, z to ETRS lat, lon, height:

In [301]:
def geographic_from_xyz(x_m, y_m, z_m, toDegrees=True):
    '''
    Compute lon, lat, and height. Based on code in lofargeo from Michiel Brentjens
    '''
    from numpy import arctan2, sqrt, cos, sin, rad2deg
    
    wgs84_f = 1./298.257223563
    
    def normalized_earth_radius(latitude_rad):
        return 1.0/sqrt(cos(latitude_rad)**2 + ((1.0 - wgs84_f)**2)*(sin(latitude_rad)**2))
        
    wgs84_a = 6378137.0

    wgs84_e2 = wgs84_f*(2.0 - wgs84_f)
    
    lon_rad = arctan2(y_m, x_m)
    r_m = sqrt(x_m**2 + y_m**2)
    # Iterate to latitude solution
    phi_previous = 1e4
    phi = arctan2(z_m, r_m)
    while np.any(abs(phi-phi_previous) > 1.6e-12):
        phi_previous = phi
        phi = arctan2(z_m + wgs84_e2*wgs84_a*normalized_earth_radius(phi)*sin(phi), r_m)
    lat_rad = phi
    height_m = r_m*cos(lat_rad) + z_m*sin(lat_rad) - wgs84_a*sqrt(1.0 - wgs84_e2*sin(lat_rad)**2)
    if toDegrees:
        return [rad2deg(lon_rad), rad2deg(lat_rad), height_m]
    else:
        return [lon_rad, lat_rad, height_m]

In [302]:
geographic_from_xyz(1,2,3)

[63.43494882292201, 89.997009702026972, -6356749.3141868291]

In [306]:
data.iloc[4138:4145]

,NAME,ETRS-X,ETRS-Y,ETRS-Z,STATION-P,STATION-Q,STATION-R,RCU-X,RCU-Y,STATIONNAME
4138,L4,3796315.727,877606.926,5032763.297,17.230,11.890,0.0,8,9,DE604
4139,L5,3796310.908,877605.628,5032767.246,16.515,18.214,0.0,10,11,DE604
4140,L6,3796316.577,877614.898,5032761.081,25.069,9.109,0.0,12,13,DE604
4141,L7,3796312.631,877614.984,5032764.090,25.628,14.040,0.0,14,15,DE604
4142,L8,3796336.904,877573.198,5032753.682,-18.907,-7.412,0.0,16,17,DE604
4143,L9,3796342.493,877571.969,5032749.638,-20.802,-14.158,0.0,18,19,DE604
4144,L10,3796343.107,877568.001,5032749.946,-24.829,-14.074,0.0,20,21,DE604


In [316]:
[data["ETRS-LON"], data["ETRS-LAT"], data["ETRS-HEIGHT"]] = geographic_from_xyz(data["ETRS-X"], data["ETRS-Y"], data["ETRS-Z"], toDegrees=True)

In [76]:
for coordpart in ['lon_rad', 'lat_rad', 'height_m']:
    data["ETRS_"+coordpart.upper().split("_")[0]] = \
          data.apply(lambda row: 
                       np.rad2deg(
                         lofargeo.geographic_from_xyz(
                           (row['ETRS-X'],row['ETRS-Y'],row['ETRS-Z'])
                         )[coordpart]
                       ), axis=1)

In [30]:
def getcoord(stationname, num):
    """
    Get the lat-lon coordinates of an HBA-antenna
    """
    queryres = data[(data["STATIONNAME"]==stationname) & (data["NUMBER"]==num) & (data["TYPE"]=="HBA")]
    if len(queryres) != 1:
        raise Exception("Problem finding antenna "+str(num)+" for station"+stationname+": found "+len(queryres))
    return np.array([queryres["ETRS_LON"].iloc[0], queryres["ETRS_LAT"].iloc[0]])

Get the x and y direction in a station

In [31]:
def numbelowfirst(stationtype):
    """ Get the antenna number for when the number in a line is below zero.
    Stationtype must be CS, RS or other"""
    if stationtype=='CSHBA0':
        return 3
    if stationtype=='CSHBA1':
        return 27
    if stationtype=='RS':
        return 4
    else:
        return 6

In [32]:
dirxs = {}
dirys = {}
for stationname in pd.unique(data["STATIONNAME"]):
    subtypes = [""]
    if stationname[0:2] == "CS":
        subtypes = ["HBA0", "HBA1"]
    for subtype in subtypes:
        firstantenna = 0
        if subtype == "HBA1":
            firstantenna = 24;
        ant0coord = getcoord(stationname, firstantenna)
        dirxs[stationname+subtype] = getcoord(stationname, firstantenna+1) - ant0coord
        highy = numbelowfirst(stationname[0:2]+subtype)
        dirys[stationname+subtype] = getcoord(stationname, highy) - ant0coord

In [33]:
data["ETRS_WKT"] = ""
for index, rec in data.iterrows():
    if rec['TYPE']!='HBA':
        continue
        
    xy=np.array([rec['ETRS_LON'],rec['ETRS_LAT']])
    
    # Find the offset vector between tiles
    subtype = ""
    if rec['STATIONNAME'][0:2] == "CS":
        if rec['NUMBER'] >= 24:
            subtype = "HBA1"
        else:
            subtype = "HBA0"
            
    dirx = dirxs[rec['STATIONNAME'] + subtype]
    diry = dirys[rec['STATIONNAME'] + subtype]

    ul = xy - 0.5*dirx + 0.5*diry
    ur = xy + 0.5*dirx + 0.5*diry 
    lr = xy + 0.5*dirx - 0.5*diry 
    ll = xy - 0.5*dirx - 0.5*diry 

    wkt=('Polygon (('+str(ul[0])+' '+str(ul[1])+', '+ 
                      str(ur[0])+' '+str(ur[1])+', '+ \
                      str(lr[0])+' '+str(lr[1])+', '+ \
                      str(ll[0])+' '+str(ll[1])+', '+ \
                      str(ul[0])+' '+str(ul[1])+'))')
    
    data.set_value(index, "ETRS_WKT", wkt)

In [34]:
a=data[data["ETRS_WKT"]!=""]["ETRS_WKT"].iloc[0]

In [36]:
a[10:].split(', ')

['6.86861583228 52.9117170256',
 '6.86868577076 52.9116982185',
 '6.86871689402 52.9117404904',
 '6.86864695554 52.9117592976',
 '6.86861583228 52.9117170256))']

In [48]:
from sqlalchemy import create_engine
import sqlalchemy

In [43]:
engine = create_engine('postgresql://dijkema@localhost:5432/lofargeo')

In [44]:
from geoalchemy2 import Geometry

In [49]:
meta = sqlalchemy.MetaData(engine)
my_table = sqlalchemy.Table('my_table', meta,
    sqlalchemy.Column('id', sqlalchemy.Integer, primary_key=True),
    sqlalchemy.Column('point', Geometry('Point', srid=4326)),
    sqlalchemy.Column('datetime', sqlalchemy.DateTime),
    sqlalchemy.Column('value', sqlalchemy.Float)
)
my_table.create(engine)

In [62]:
data.to_csv("allantennas.csv")

In [66]:
engine.execute("delete from stations;")

In [60]:
stations = pd.read_excel("/Users/dijkema/Desktop/lofargeo/stationsnamen.xlsx")

In [61]:
stations.to_sql('stations', engine, index=False, if_exists='append')